# ダイクストラ法

In [2]:
# 隣接リストに辺の重みを格納
# 各ノードへの最短距離をinfで初期化
# 始点を最短距離0で確定
# 始点から一手で行けるノードに仮の最短距離を入れる

# 未確定のノードの内一番近い点を確定、そこから一手でいけるノードに仮の最短距離を入れる
# ↑を全部の点が確定するまで繰り返し

In [15]:
# ノード数、エッジ数、始点
v,e,r=map(int,input().split())

# 隣接リスト a[i]には[iから行ける辺,コスト]のリストが格納される(三次元配列)
a = [[] for _ in range(v)]
for _ in range(e):
    s,t,d=map(int,input().split())
    a[s].append([t,d])


4 5 0
0 1 1
0 2 4
1 2 2
2 3 1
1 3 5


In [21]:
from heapq import heappush, heappop

# ダイクストラ法
def dijkstra(s,n): #始点, ノード数
    dist=[float("inf")]*n
    q=[[0,s]] # 始点をキューに追加
    dist[s]=0
    check=[False]*n # 最小コストが確定した点はTrueになる
    while q:
        # ノードをpop
        # ヒープの頭をpopするので未確定のノードのなかで最小コストの点が出てくる
        v = heappop(q)[1] 
        check[v]=True
        for to,cost in a[v]:
            # 行先が未確定で最短コストを更新できそうなら行先を探索キューに追加
            if check[to]==False and dist[to]>dist[v]+cost:
                dist[to]=dist[v]+cost
                heappush(q,[dist[to],to])
    return dist

d=dijkstra(r,v)

for ans in d:
    print("INF" if ans==float("inf") else ans)

# F - 船旅

https://atcoder.jp/contests/joi2008yo/tasks/joi2008yo_f

#### 注文票が来るたびにダイクストラ使うだけ

In [ ]:
from heapq import heappush, heappop

# ダイクストラ法
def dijkstra(s,n): #始点, ノード数
    dist=[float("inf")]*n
    q=[[0,s]] # 始点をキューに追加
    dist[s]=0
    check=[False]*n # 最小コストが確定した点はTrueになる
    while q:
        # ノードをpop
        # ヒープの頭をpopするので未確定のノードのなかで最小コストの点が出てくる
        v = heappop(q)[1] 
        check[v]=True
        for to,cost in a[v]:
            # 行先が未確定で最短コストを更新できそうなら行先を探索キューに追加
            if check[to]==False and dist[to]>dist[v]+cost:
                dist[to]=dist[v]+cost
                heappush(q,[dist[to],to])
    return dist


n,k = map(int,input().split())
# 隣接リスト
a = [[] for _ in range(n+1)] # 島の番号が1始まりなので一つ増やした

for _ in range(k):
    o = [int(i) for i in input().split()]
    if o[0]==1:
        a[o[1]].append([o[2],o[3]])
        a[o[2]].append([o[1],o[3]])
    elif o[0]==0:
        dist = dijkstra(o[1],n+1)
        print(-1 if dist[o[2]]==float("inf") else dist[o[2]])

# E - ゾンビ島 (Zombie Island) 

貼り付け元  <https://atcoder.jp/contests/joi2016yo/tasks/joi2016yo_e> 

#### 1.幅優先探索で危険な町を判定
#### 2.町が危険かどうかに応じてコストを設定して、ダイクストラ用の重み付きパスを作る
#### 3.ダイクストラで全ての町へのコストを調べる　最後から一歩前の町へのコストの最小値が答え

In [74]:
import numpy as np
from copy import copy
from heapq import heappush, heappop

N,M,K,S = map(int,input().split())
P,Q = map(int,input().split())

zombie_town=[int(input()) for _ in range(K)]

path_for_bfs = [[] for _ in range(N+1)]

for _ in range(M):
    path = [int(i) for i in input().split()]
    path_for_bfs[path[0]].append(path[1])
    path_for_bfs[path[1]].append(path[0])


21 26 2 2
1000 2000
5 
16
1 2
1 3
1 10
2 5
3 4
4 6
5 8
6 7
7 9
8 10
9 10
9 11
11 13
12 13
12 15
13 14
13 16
14 17
15 16
15 18
16 17
16 19
17 20
18 19
19 20
19 21


In [ ]:
def get_danger(p_start):
    dist=[0]*(N+1)
    dist[p_start]=-1 #あとで引っかからないように0じゃない値にしておく
    que=path_for_bfs[p_start]
    
    for cnt in range(1,S+1):
        p_add=[]
        for p_from in que:
            if dist[p_from]==0:
                dist[p_from]=cnt
                p_add+=path_for_bfs[p_from]
        que=list(set(p_add))
    dist[p_start]=0
    # 危険フラグを返す
    return np.array(dist)>0
  
# 危険な町を取得
danger_town=np.array([False]*(N+1))
for town in zombie_town:
    danger_town = danger_town + get_danger(town)


# ゾンビがいない町の宿泊コストを格納
path_for_dijkstra = [[] for _ in range(N+1)]
for p_from in range(1,N+1):
    for to in path_for_bfs[p_from]:
        # ゾンビがいる町は追加しない
        if to in zombie_town:
            continue            
        # 危険かどうかに応じて宿泊コストを詰め込む
        if danger_town[to]==True:
            path_for_dijkstra[p_from].append([to,Q])
        else:
            path_for_dijkstra[p_from].append([to,P])

            
# ダイクストラ法（distには最小コストが格納されてる）
def dijkstra(s,n): #始点, ノード数
    lowest=[float("inf")]*n
    q=[[0,s]] # 始点をキューに追加
    lowest[s]=0
    check=[False]*n # 最小コストが確定した点はTrueになる
    while q:
        # ノードをpop
        # ヒープの頭をpopするので未確定のノードのなかで最小コストの点が出てくる
        v = heappop(q)[1] 
        check[v]=True
        for to,cost in path_for_dijkstra[v]:
            # 行先が未確定で最短コストを更新できそうなら行先を探索キューに追加
            if check[to]==False and lowest[to]>lowest[v]+cost:
                lowest[to]=lowest[v]+cost
                heappush(q,[lowest[to],to])
    return lowest


lowest = dijkstra(1,N+1)
ans=float("inf")
# 最後の宿は泊まらなくていいので、一歩手前で一番安いところを探す
for i in path_for_dijkstra[N]:
    ans=min(ans,lowest[i[0]])
print(ans)